# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pickle
import torch

from agent import Agent, BATCH_SIZE, BUFFER_SIZE

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe")
# env = UnityEnvironment(file_name="Banana_Linux_NoVis/Banana.x86_64")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    break
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
from models.model_64 import QNetwork
from replay_buffer import ProportionalReplayBuffer, RankBasedReplayBuffer, UniformReplayBuffer

# TODO: create a real seed
seed = 0

model = QNetwork

memory = UniformReplayBuffer(action_size, BATCH_SIZE, BUFFER_SIZE, seed)
# memory = ProportionalReplayBuffer(action_size, BATCH_SIZE, BUFFER_SIZE, e=0.2, alpha=0.5, beta0=0.1, beta_inc=0.02, seed=seed)
# memory = RankBasedReplayBuffer(action_size, BATCH_SIZE, BUFFER_SIZE, buffer_type='array', alpha=0.5, beta0=0.1, beta_inc=0.02, seed=seed)
agent = Agent(state_size, action_size, model, memory, seed=0)

In [ ]:
checkpoints_folder = 'checkpoints'

def save_model(model, suffix=''):
    file_name = f'checkpoint_{model_name}'
    if suffix:
        file_name += f'_{suffix}'
    file_name = f'{file_name}.pth' # add file extension
    file_path = os.path.join(checkpoints_folder, file_name)
    torch.save(model.state_dict(), file_path)
def save_scores(scores):
    file_name = f'score_{model_name}.pkl'
    file_path = os.path.join(checkpoints_folder, file_name)
    pickle.dump(scores, open(file_path, 'wb'))
def step_env(action):
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0] 
    done = env_info.local_done[0]  
    return (next_state, reward, done)

In [ ]:
model_name = 'dqn_64'

def dqn(n_episodes=2000, eps_start=1.0, eps_end=0.01, eps_decay=0.995, save_scores_every=50):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    # the target score after each episode.
    target_score = 13
    
    # list containing scores from each episode.
    scores = []
    
    # the number of consecultive times that the target score has been reached.
    reached_score_times = 0
    
    # last 100 scores
    scores_window = deque(maxlen=100)
    
    # initialize epsilon
    eps = eps_start
    
    # if the model achieved the goal for the first time or not.
    reached_goal = False
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        done = False
        while not done:
            action = agent.act(state, eps)
            next_state, reward, done = step_env(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        
        eps = max(eps_end, eps_decay**2*eps) # decrease epsilon

        print('\rEpisode {}\tScore: {:.2f}\tAverage Score: {:.2f}\tEpsilon: {:.2f}'.format(i_episode, score, np.mean(scores_window), eps))
            
        if i_episode % save_scores_every == 0:
            save_scores(scores)
        
        if np.mean(scores_window) > 13 and not reached_goal:
            reached_goal = True
            save_model(agent.qnetwork_local, 'goal')
            print(f'Achieved goal on the episode {i_episode}')
    
    save_model(agent.qnetwork_local)
    save_scores(scores)
    
    return scores

scores = dqn()

In [ ]:
# plot the scores.
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
# plot some options for epsilon decay.

eps_start=1.0
eps_end=0.01
eps_decay=0.995


eps_1 = lambda i_episode: max(eps_end, eps_decay**(i_episode)) # decrease epsilon
eps_5 = lambda i_episode: max(eps_end, eps_decay**(2*i_episode)) # decrease epsilon
eps_2 = lambda i_episode: max(eps_end, eps_start-(eps_start-eps_end)/250*i_episode)
eps_4 = lambda i_episode: max(eps_end, eps_start-(eps_start-eps_end)/50*i_episode)
eps_3 = lambda i_episode: max(eps_end, eps_start/i_episode)

episodes = range(1, 1000)

r = []
eps = eps_start
for i_episode in episodes:
    r.append(eps)
    eps = max(eps_end, eps_decay**2*eps)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(episodes, [eps_1(i) for i in episodes], label='Exp')
plt.plot(episodes, [eps_5(i) for i in episodes], label='Exp2')
plt.plot(episodes, r, label='ExpM')
plt.plot(episodes, [eps_2(i) for i in episodes], label='Mod1')
plt.plot(episodes, [eps_4(i) for i in episodes], label='Mod2')
plt.plot(episodes, [eps_3(i) for i in episodes], label='Linear')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend()
plt.show()